## wconcept 추천 시스템

In [1]:
# 검색어 '바라클라바' 추천순 제품 10개 댓글 긍부정 분석
# 제품 10개 각각 하단 you may also like 상품 세개~다섯개 댓글 긍부정 확인
# -> 각각의 긍정키워드 / 부정키워드 분석 (긍부정 비율 + 긍정 이유 및 부정 이유의 유사성 확인)

# -> 추천 시스템 작동 원리 확인...?

# 데이터 수집

In [2]:
# 크롤링

# url = 'product'

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import WebDriverWait

service = Service()
options = webdriver.ChromeOptions()

page = 1

# url = "https://www.e-himart.co.kr/app/display/showDisplayCategory?dispNo=1011030000#pageCount={}".format(page)
url = 'https://display.wconcept.co.kr/search?kwd=%EB%B0%94%EB%9D%BC%ED%81%B4%EB%9D%BC%EB%B0%94&sort=WCK&type=popular'
driver = webdriver.Chrome(service=service,options=options)
driver.get(url)

In [3]:
# url = 'https://display.wconcept.co.kr/search?kwd=%EB%B0%94%EB%9D%BC%ED%81%B4%EB%9D%BC%EB%B0%94&sort=WCK&type=popular'
# driver = webdriver.Chrome(service=service,options=options)
# driver.get(url)

# driver.execute_script("window.scrollTo(0, 1000);")

contents = driver.find_element(By.ID,"product")
products = contents.find_elements(By.CLASS_NAME,"product-item")     # 조금 더 좁히면 class="area-info"

companies = []        # 회사명              # class="sc-tdjdw3-0 frpBAf text title"
prdNames = []         # 제품명              # class="sc-tdjdw3-0 frpBAf text detail"
prdPrices = []        # 가격                # 가격섹션 (sc-jpuob9-3 iEzjpB prdc-price) 정가(sc-tdjdw3-0 frpBAf text customer-price) 할인율(sc-tdjdw3-0 frpBAf text final-discount) 최종할인가(sc-tdjdw3-0 frpBAf text final-price)
prdRatings = []       # 별점                # class="score"
prdWish = []          # 관심상품             # class="sc-tdjdw3-0 frpBAf text cnt"

btn_product = ''      # 상품 상세로 이동       # class="sc-16sqvmz-0 hajYLp btn sc-jpuob9-7 ertTIN area-click"
btn_review = ''       # 상품 리뷰로 이동        # ul id="detail" - selector #detail > li:nth-child(2) > a
btn_list = ''         # 상품 리스트로 이동      #


sec = 0.5
time.sleep(sec)

try:
    for idx, product in enumerate(products):
        # companies.append(product.find_element(By.CLASS_NAME,"title").text)
        # prdNames.append(product.find_element(By.CLASS_NAME,"detail").text)
        # prdPrices.append(product.find_element(By.CLASS_NAME,"final-price").text)
        # prdRatings.append(product.find_element(By.CLASS_NAME,"score").text)
        # prdWish.append(product.find_element(By.CLASS_NAME,"cnt").text)
        
        btn_product = product.find_element(By.CLASS_NAME,"area-click")
        btn_product.click()
        time.sleep(sec)
        driver.execute_script("window.scrollTo(0, 1500);")
        btn_review = driver.find_element(By.CSS_SELECTOR,"#detail > li:nth-child(2) > a")
        btn_review.click()

except Exception as e:  
    pass


In [11]:
url = "https://www.wconcept.co.kr/Product/303885631?entry_channel=search&rccode=mo_search&search_menu=ranking#review"
driver = webdriver.Chrome(service=service,options=options)
driver.get(url)

reviews = []
driver.execute_script("window.scrollTo(0, 1500);")
btn_review = driver.find_element(By.CSS_SELECTOR,"#detail > li:nth-child(2) > a")
btn_review.click()

for i in range(0,10):
    
    position = driver.find_element(By.CLASS_NAME,"pdt_review_table")
    driver.execute_script("arguments[0].scrollIntoView();", position)
    
    
    review_list = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,"reviewList")))
    review_list = driver.find_element(By.ID,"reviewList") # -> 해당 페이지의 리뷰 모두 나옴
    review_info = review_list.find_elements(By.CLASS_NAME,"pdt_review_info")
    review_grade = review_list.find_elements(By.TAG_NAME,"strong")
    review_detail = review_list.find_elements(By.CLASS_NAME,"pdt_review_text")
    


    for info, grade, detail in zip(review_info, review_grade, review_detail):
        review = [info.text, grade.get_attribute('style'), detail.text]
        reviews.append(review)

    time.sleep(sec)
    
    pages = WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.CLASS_NAME,"pagination")))
    driver.execute_script("arguments[0].scrollIntoView();", pages)
    btns = pages.find_elements(By.TAG_NAME,"li")
    btn_next = btns[i+3]
    btn_next.click()


ElementClickInterceptedException: Message: element click intercepted: Element <li>...</li> is not clickable at point (504, 11). Other element would receive the click: <a href="/BRAND" onclick="GA4EventPush('gnb_click', '...', 'gnb', 'gnb_list_tab', '7'); AMPLITUDE.LogClickGnbHeader(1,'click_gnb','gnb_no7', 'DESIGNER', '7','/BRAND');">DESIGNER</a>
  (Session info: chrome=120.0.6099.129)
Stacktrace:
0   chromedriver                        0x000000010618fcc8 chromedriver + 4844744
1   chromedriver                        0x0000000106187243 chromedriver + 4809283
2   chromedriver                        0x0000000105d5577d chromedriver + 411517
3   chromedriver                        0x0000000105dae522 chromedriver + 775458
4   chromedriver                        0x0000000105dabcb6 chromedriver + 765110
5   chromedriver                        0x0000000105da8f57 chromedriver + 753495
6   chromedriver                        0x0000000105da7a6b chromedriver + 748139
7   chromedriver              

In [9]:
reviews

[['구매옵션 : grey\nja******** 2023.12.27',
  'width: 100%;',
  '가격이 좀 비싸긴했는데 그만큼 소재도\n도톰하고 따뜻할거같아요\n더세일을 하네요 아쉽지만 색상이랑 다마음에\n들어요'],
 ['구매옵션 : grey\nrl******** 2023.12.27',
  'width: 100%;',
  '친구 선물로 줫는데 아주좋아합니당 ㅎㅎ 잘쓸게요'],
 ['구매옵션 : charcoal\nja******** 2023.12.27',
  'width: 100%;',
  '얼굴에 달라붙지 않고 여유 있어서 부담스럽지않아요\n맨투맨 니트조끼 패팅 어디에나 희뚜루 마뚜루 템'],
 ['구매옵션 : beige\nqk******** 2023.12.27', 'width: 100%;', '귀여워요'],
 ['구매옵션 : black\nhe******** 2023.12.27',
  'width: 100%;',
  '보풀이 좀 잘 일어나는 소재지만 따뜻해서 좋아요'],
 ['구매옵션 : grey\nwc******** 2023.12.27',
  'width: 100%;',
  '처음받았을때는 이상한 냄새가나는게 쫌 안좋았지만 그래도 이쁘고 따뜻해서 잘 쓰고있습니다\n무난하고 좋아요'],
 ['구매옵션 : grey\nja******** 2023.12.27',
  'width: 100%;',
  '가격이 좀 비싸긴했는데 그만큼 소재도\n도톰하고 따뜻할거같아요\n더세일을 하네요 아쉽지만 색상이랑 다마음에\n들어요'],
 ['구매옵션 : grey\nrl******** 2023.12.27',
  'width: 100%;',
  '친구 선물로 줫는데 아주좋아합니당 ㅎㅎ 잘쓸게요'],
 ['구매옵션 : charcoal\nja******** 2023.12.27',
  'width: 100%;',
  '얼굴에 달라붙지 않고 여유 있어서 부담스럽지않아요\n맨투맨 니트조끼 패팅 어디에나 희뚜루 마뚜루 템'],
 ['구매옵션 : beige\nqk******** 202

# 성공 코드 모아놓기

In [ ]:
# 상품 정보만 크롤링하기
# contents = driver.find_element(By.ID,"product")
# products = contents.find_elements(By.CLASS_NAME,"product-item")     # 조금 더 좁히면 class="area-info"

# companies = []        # 회사명        # sc-tdjdw3-0 frpBAf text title
# prdNames = []         # 제품명        # sc-tdjdw3-0 frpBAf text detail
# prdPrices = []        # 가격          # 가격섹션 (sc-jpuob9-3 iEzjpB prdc-price) 정가(sc-tdjdw3-0 frpBAf text customer-price) 할인율(sc-tdjdw3-0 frpBAf text final-discount) 최종할인가(sc-tdjdw3-0 frpBAf text final-price)
# prdRatings = []       # 별점          # score
# prdWish = []          # 관심상품      # sc-tdjdw3-0 frpBAf text cnt
# sec = 0.5
# time.sleep(sec)

# try:
#     for idx, product in enumerate(products):
#         companies.append(product.find_element(By.CLASS_NAME,"title").text)
#         prdNames.append(product.find_element(By.CLASS_NAME,"detail").text)
#         prdPrices.append(product.find_element(By.CLASS_NAME,"final-price").text)
#         prdRatings.append(product.find_element(By.CLASS_NAME,"score").text)
#         prdWish.append(product.find_element(By.CLASS_NAME,"cnt").text)
        
        
#         if idx > 10:
#             break
# except Exception as e:  
#     pass


In [ ]:
# 리뷰페이지까지 입성

# url = 'https://display.wconcept.co.kr/search?kwd=%EB%B0%94%EB%9D%BC%ED%81%B4%EB%9D%BC%EB%B0%94&sort=WCK&type=popular'
# driver = webdriver.Chrome(service=service,options=options)
# driver.get(url)

# contents = driver.find_element(By.ID,"product")
# products = contents.find_elements(By.CLASS_NAME,"product-item")     # 조금 더 좁히면 class="area-info"

# companies = []        # 회사명              # class="sc-tdjdw3-0 frpBAf text title"
# prdNames = []         # 제품명              # class="sc-tdjdw3-0 frpBAf text detail"
# prdPrices = []        # 가격                # 가격섹션 (sc-jpuob9-3 iEzjpB prdc-price) 정가(sc-tdjdw3-0 frpBAf text customer-price) 할인율(sc-tdjdw3-0 frpBAf text final-discount) 최종할인가(sc-tdjdw3-0 frpBAf text final-price)
# prdRatings = []       # 별점                # class="score"
# prdWish = []          # 관심상품             # class="sc-tdjdw3-0 frpBAf text cnt"

# btn_product = ''      # 상품 상세로 이동       # class="sc-16sqvmz-0 hajYLp btn sc-jpuob9-7 ertTIN area-click"
# btn_review = ''       # 상품 리뷰로 이동        # ul id="detail" - selector #detail > li:nth-child(2) > a
# btn_list = ''         # 상품 리스트로 이동      #


# sec = 0.5
# time.sleep(sec)

# try:
#     for idx, product in enumerate(products):
#         # companies.append(product.find_element(By.CLASS_NAME,"title").text)
#         # prdNames.append(product.find_element(By.CLASS_NAME,"detail").text)
#         # prdPrices.append(product.find_element(By.CLASS_NAME,"final-price").text)
#         # prdRatings.append(product.find_element(By.CLASS_NAME,"score").text)
#         # prdWish.append(product.find_element(By.CLASS_NAME,"cnt").text)
        
#         btn_product = product.find_element(By.CLASS_NAME,"area-click")
#         btn_product.click()
#         time.sleep(sec)
#         driver.execute_script("window.scrollTo(0, 1500);")
#         btn_review = driver.find_element(By.CSS_SELECTOR,"#detail > li:nth-child(2) > a")
#         btn_review.click()
 

#         if idx > 10:
#             break
# except Exception as e:  
#     pass
